<a href="https://colab.research.google.com/github/benihime91/pytorch_retinanet/blob/master/003_udacity_self_driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# What GPU do we have ?
!nvidia-smi

In [ ]:

# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
# Clone the RetinaNet Repo
!git clone https://github.com/benihime91/pytorch_retinanet.git
# install dependencies
!pip install pytorch-lightning omegaconf wandb --quiet
!pip install git+https://github.com/albumentations-team/albumentations --quiet
!echo "[   OK   ] Installed all depedencies "

In [ ]:
#Update sys path to enclude the pytorch RetinaNet modules
import warnings
import os
import sys

warnings.filterwarnings('ignore')
sys.path.append("/content/pytorch_retinanet/")
%load_ext autoreload
%autoreload 2
%matplotlib inline

!echo "[   OK   ] Setup Done "

In [ ]:
#Downloading data Udacity-self driving dataset from Roboflow
#UPDATE THIS LINK - get our data from Roboflow
#https://public.roboflow.com/object-detection/self-driving-car
%cd /content
!curl -L "[ROBOFLOW DATA DOWNLOAD LINK]" > roboflow.zip
!unzip -qq roboflow.zip
!rm roboflow.zip

In [ ]:
#Set up paths 

#Path to where the Images are stored
IMAGE_PATH = "/content/export"
#Path where annotations are stored
ANNOT_PATH = "/content/export"

In [ ]:
import pandas as pd
from utils.pascal import convert_annotations_to_df
from PIL import Image
import cv2
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth", None)
np.random.seed(123)


def remove_invalid_annots(df):
    """
    Removes annotations where xmax, ymax < xmin,ymin
    from the given dataframe
    """
    df = df[df.xmax > df.xmin]
    df = df[df.ymax > df.ymin]
    df.reset_index(inplace=True, drop=True)
    return df

def split_dataframe(dataframe, split_size:float=0.3, seed=123):
    """
    Splits a given pandas DataFrame object in `split_size`
    
    Args:
        dataframe: a pandas DataFrame object
        split_size: fraction size of the test dataframe
        seed : a random seed to ensure reproducibility
    
    Returns :
        1. train_df: a pandas DataFrame of size `(1-split_size)*len(dataframe)`
        2. test_df : a pandas DataFrame of size `(split_size)*len(dataframe)`
    """
    unique_ids = list(dataframe.filename.unique())
    train_ids, test_ids = train_test_split(unique_ids, 
                                           shuffle=True, 
                                           random_state=seed, 
                                           test_size=split_size
                                           )
    
    dataframe["split"] = 0
    
    for i,idx in enumerate(tqdm(dataframe.filename.values)):
        if idx in set(train_ids): 
            dataframe["split"][i] = "train"
        
        elif idx in set(test_ids) : 
            dataframe["split"][i] = "test"

    #Create the training and test dataframes
    df_train = dataframe.loc[dataframe["split"] == "train"]
    df_test = dataframe.loc[dataframe["split"] == "test"]
    #Reset indices and drop the redundant "split" column
    df_train, df_test = df_train.reset_index(drop=True),df_test.reset_index(drop=True)
    df_train.drop(columns=["split"], inplace=True)
    df_test.drop(columns=["split"], inplace=True)

    return df_train, df_test

In [ ]:
#Create a pandas dataframe object from the xml files and filter the invalid annotations
dataframe = remove_invalid_annots(convert_annotations_to_df(ANNOT_PATH, IMAGE_PATH, image_set="train"))
dataframe.head()

In [ ]:
#Create train and test datasets
train_df, test_df = split_dataframe(dataframe=dataframe, split_size=0.3)
print("Num Training examples: ", len(train_df.filename.unique()), end="\n\n")

#Create test and validation datasets
valid_df, test_df = split_dataframe(dataframe=test_df, split_size=0.5)
print("Num Training examples: ", len(test_df.filename.unique()))
print("Num Validation examples:", len(valid_df.filename.unique()))

In [ ]:
#save dataframes to memory
TRAIN_PATH = "/content/train_data.csv"
TEST_PATH = "/content/test_data.csv"
VALIDATION_PATH = "/content/validation_data.csv"


train_df.to_csv(TRAIN_PATH, index=False)
test_df.to_csv(TEST_PATH, index=False)
valid_df.to_csv(VALIDATION_PATH, index=False)

In [ ]:
from utils.pascal import generate_pascal_category_names
from utils import visualize_boxes_and_labels_on_image_array as viz_bbs
import matplotlib.pyplot as plt

#Generate a lable map for categories
LABEL_MAP = generate_pascal_category_names(dataframe)
print(LABEL_MAP)


def grab_bbs_(dataframe, index:int):
    """
    Takes in a Pandas DataFrame and a index number
    Returns filename of the image and all the bounding boxes and class_labels
    corresponding the image that is at the given index
    """
    assert index <= len(dataframe), f"[  ERROR  ] Invalid index for dataframe with len: {len(dataframe)}"
    fname = dataframe.filename[index]
    locs  = dataframe.loc[dataframe.filename == fname]
    bbs   = locs[["xmin", "ymin", "xmax", "ymax"]].values
    cls   = locs["labels"].values
    return fname, bbs, cls

def load_image_from_data(dataframe, index):
    """
    Loads in a image from the given dataframe at given index
    Returns a PIL image object contraining all the bounding boxes over
    the image
    """
    image, boxes, clas = grab_bbs_(dataframe, index)
    #load and normalize the image
    image = Image.open(image)
    image = np.array(image) / 255.
    image = viz_bbs(image, boxes, scores=None, classes=clas, label_map=LABEL_MAP)
    return image

In [ ]:
image = load_image_from_data(train_df, index=10)

plt.figure(figsize=(15,15))
plt.imshow(image)
plt.axis("off")
plt.show();

In [ ]:
image = load_image_from_data(test_df, index=100)

plt.figure(figsize=(15,15))
plt.imshow(image)
plt.axis("off")
plt.show();

In [ ]:
from omegaconf import OmegaConf

hparams = OmegaConf.load("/content/pytorch_retinanet/hparams.yaml")

hparams.dataset.kind = "csv"

hparams.dataset.trn_paths   = TRAIN_PATH
hparams.dataset.valid_paths = VALIDATION_PATH
hparams.dataset.test_paths  = TEST_PATH
hparams.model.num_classes = len(LABEL_MAP) - 1

hparams.optimizer = {
    "class_name": "torch.optim.SGD", 
    "params": {
        "lr": 0.003,
        "momentum": 0.9,
        "weight_decay" : 0.001,
        },
    }

print(OmegaConf.to_yaml(hparams))

In [ ]:
import time
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import *
from pytorch_lightning.callbacks import *
from model import RetinaNetModel, LogCallback

# seed so that results are reproducible
pl.seed_everything(123)

In [ ]:
#Logger
#wandb API-KEY
!wandb login "[WANDB API-KEY]"
#Wandb project name
PNAME = "[WANDB PROJECT-NAME]"
LOGGER = WandbLogger(name=f"{time.time()}", anonymous=True, project=PNAME)

#Learning-rate Logger to log the learning-rate to the logger
LR_LOGGER = LearningRateLogger(logging_interval="step")

#Model Checkpoint Callback, this callback will save checkpoints 
#each time our val loss decreases
fname =f"/content/checkpoints/"
os.makedirs(fname, exist_ok=True)
CHECKPOINT_CALLBACK = ModelCheckpoint(fname, mode="min", monitor="val_loss", save_top_k=3,)

#callback for early-stopping
EARLY_STOPPING_CALLBACK = EarlyStopping(mode="min", monitor="val_loss", patience=12, verbose=True)

#instantiate LightningTrainer
trainer = Trainer(
    precision=16, 
    max_epochs=60,
    gpus=1, 
    logger=[LOGGER],
    early_stop_callback=EARLY_STOPPING_CALLBACK, 
    checkpoint_callback=CHECKPOINT_CALLBACK,
    callbacks=[LogCallback(), LR_LOGGER], 
    weights_summary=None,
    terminate_on_nan=True, 
    benchmark=True,
    );

In [ ]:
#Instantiate lightning-module
litModel = RetinaNetModel(hparams=hparams)
#Start Train
trainer.fit(litModel)

In [ ]:
#Evaluations results on the test/ validation dataset(if test dataset is not given)
#using COCO API
trainer.test()

In [ ]:
import torch
from retinanet import Retinanet

PATH = f"/content/trained_weights.pth"
torch.save(litModel.model.state_dict(), PATH)

state_dict = torch.load(PATH)
MODEL = Retinanet(num_classes=hparams.model.num_classes, backbone_kind=hparams.model.backbone_kind)
MODEL.load_state_dict(state_dict)
MODEL.to("cuda:0");

In [ ]:
from PIL import Image
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

@torch.no_grad()
def get_preds(path, threshold=0.6,):
    """
    Generates predictions on the given image from the given path.
    """
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    
    INFER_TRANSFORMS = A.Compose([A.ToFloat(max_value=255.0, always_apply=True),
                                  ToTensorV2(always_apply=True)
                                  ])
    
    TENSOR_IMAGE = INFER_TRANSFORMS(image=image)["image"].to("cuda:0")
    PREDICTIONS = MODEL.predict([TENSOR_IMAGE])
    return PREDICTIONS[0]

def detect(image_path, threshold=0.6):
    """
    Generate detections on the image that is present in 
    the given image path

    Args:
        image_path: Path to the input Image
        threshold: Score threshold to filter predictions
        nms_threshold: NMS threshold

    Returns: a PIL image containg the original Image and
             bounding boxes draw over it.
    """
    # Generate predictions for the given image
    preds = get_preds(image_path, threshold,)
    # print(preds)
    # Filter predictions
    boxes, labels, scores = preds["boxes"], preds["labels"], preds["scores"]
    mask   = scores > threshold
    boxes  = boxes[mask]
    labels = labels[mask]
    scores = scores[mask]
    return boxes.cpu().numpy(), labels.cpu().numpy(), scores.cpu().numpy()

def draw_on_image(image_path, boxes, scores, classes, label_map=LABEL_MAP):
    """
    Draw bounding box over the image at image path, with the scores and classes
    Returns a PIL image object.
    
    Args: 
        image_path `(str)`: Path to the input Image
        boxes `(List[N,4])`: absolute bouding box co-ordiates in the form `[xmin,ymin,xmax,ymax]`.
        scores `(List[N])` : List containing the scores for each of the bounding box.
        classes `(`List[N])`: List containing the class_labels for each of the bounding box.
        label_map `(List[num_classes])`: List of the labels

    Returns: 
        A PIL image object
    """
    image = Image.open(image_path)
    image = np.array(image) / 255.
    image = viz_bbs(image, boxes, scores=scores, classes=classes, label_map=LABEL_MAP)
    return image

In [ ]:
#Path to the image
image_path = test_df.filename[100]
#generate predictions for the image
boxes, labels, scores = detect(image_path, threshold=0.5)

image = draw_on_image(image_path, boxes, scores, labels)

plt.figure(figsize=(15,15))
plt.imshow(image)
plt.axis("off")
plt.show();